## Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors

## Limpeza e pré processamento dos dados

### Carregamento

Apenas colunas que serão utilizadas na análise

In [ ]:
desired_cols = [
  "ID_AGRAVO",
  "DT_NOTIFIC", # Início do caso 
  "DT_SIN_PRI",

  "ANO_NASC",  # Info pessoal
  "CS_SEXO",
  "CS_GESTANT",
  "CS_RACA",
  "CS_ESCOL_N",
  "SG_UF",
  "ID_MN_RESI",
  "ID_RG_RESI",

  "FEBRE",    # Sintomas
  "MIALGIA",
  "CEFALEIA",
  "EXANTEMA",
  "VOMITO",
  "NAUSEA",
  "DOR_COSTAS",
  "CONJUNTVIT",
  "ARTRITE",
  "ARTRALGIA",
  "PETEQUIA_N",
  "LEUCOPENIA",
  "LACO",
  "DOR_RETRO",

  "DIABETES",   # Comorbidades
  "HEMATOLOG",
  "HEPATOPAT",
  "RENAL",
  "HIPERTENSA",
  "ACIDO_PEPT",
  "AUTO_IMUNE",

  "HOSPITALIZ", # Desfecho
  "CLASSI_FIN", 
  "EVOLUCAO",
  "DT_ENCERRA"
]

In [ ]:
# Carregando dataset
DATA_PATH = "./data/original-2/DENGBR25.csv"
df = pd.read_csv(DATA_PATH, low_memory=False, usecols=desired_cols)

In [ ]:
df.head()

In [ ]:
df.info()

### Identificação e remoção de dados inconsistentes para colunas que participarão da clusterização

#### ANO_NASC

In [ ]:
df["ANO_NASC"].unique()

In [ ]:
df = df.dropna(subset=["ANO_NASC"])
df["ANO_NASC"].isna().sum()

#### CS_SEXO

In [ ]:
df["CS_SEXO"].unique()

In [ ]:
df["CS_SEXO"] = df["CS_SEXO"].replace("I", np.nan)
df = df.dropna(subset=["CS_SEXO"])
df["CS_SEXO"].unique()

#### CS_GESTANT

1 - 1º Trimestre <br>
2 - 2º Trimestre <br>
3 - 3º Trimestre <br>
4 - Idade gestacional ignorada <br>
5 - Não <br>
6 - Não se aplica <br>
9 - Ignorado <br>

In [ ]:
df["CS_GESTANT"].unique()

In [ ]:
df["CS_GESTANT"] = df["CS_GESTANT"].replace(9.0, np.nan)
df = df.dropna(subset=["CS_GESTANT"])
df["CS_GESTANT"].unique()

#### CS_RACA

1- branca <br>
2- preta <br>
3- amarela <br>
4- parda <br>
5- indígena <br>
9- Ignorado <br>

In [ ]:
df["CS_RACA"].unique()

In [ ]:
df["CS_RACA"] = df["CS_RACA"].replace(9.0, np.nan)
df = df.dropna(subset=["CS_RACA"])
df["CS_RACA"].unique()

#### CS_ESCOL_N

Ign/Branco
,9 ,09,99 <br>
Analfabeto
0 ,00 <br>
1ª a 4ª série incompleta do EF
1
,01 <br>
4ª série completa do EF
2
,02 <br>
5ª a 8ª série incompleta do EF 3 ,03 <br>
Ensino fundamental completo 4 ,04 <br>
Ensino médio incompleto
5
,05 <br>
Ensino médio completo
6
,06 <br>
Educação superior incompleta
7
,07 <br>
Educação superior completa
8 ,08 <br>
Não se aplica
10

In [ ]:
df["CS_ESCOL_N"].unique()

In [ ]:
df["CS_ESCOL_N"] = df["CS_ESCOL_N"].replace(9.0, np.nan).replace(10.0, np.nan)
df = df.dropna(subset=["CS_ESCOL_N"])
df["CS_ESCOL_N"] = df["CS_ESCOL_N"].astype(int)
df["CS_ESCOL_N"].unique()

In [ ]:
df["SG_UF"].unique()

#### Para as demais colunas, somente remover valores NaN

In [ ]:
df = df.dropna()
df.info()

### Transformação de dados

#### ANO_NASC

In [ ]:
current_year = 2025
df["IDADE"] = current_year - df["ANO_NASC"]
df["IDADE"] = df["IDADE"].astype(int)
df["IDADE"].unique()

#### CS_SEXO

In [ ]:
df_encoded_sex = pd.get_dummies(df, columns=["CS_SEXO"], drop_first=True)
df_encoded_sex["CS_SEXO_M"] = df_encoded_sex["CS_SEXO_M"].astype(int)

#### CS_GESTANT

1, 2, 3, 4 -> Sim <br>
5, 6 -> Não

In [ ]:
df_encoded_sex["CS_GESTANT"] = df_encoded_sex["CS_GESTANT"].replace([1, 2, 3, 4], "S")
df_encoded_sex["CS_GESTANT"] = df_encoded_sex["CS_GESTANT"].replace([5, 6], "N")
df_encoded_gest = pd.get_dummies(df_encoded_sex, columns=["CS_GESTANT"], drop_first=True)
df_encoded_gest["CS_GESTANT_S"] = df_encoded_gest["CS_GESTANT_S"].astype(int) 
df_encoded_gest.head()

#### CS_RACA

In [ ]:
# Trocando raça de float pra int
df_encoded_gest["CS_RACA"] = df_encoded_gest["CS_RACA"].astype(int)

# One hot encoding
df_encoded_raca = pd.get_dummies(df_encoded_gest, columns=["CS_RACA"])

# Trocando true e false para 1 e 0
colunas_bool = df_encoded_raca.select_dtypes(include='bool').columns
df_encoded_raca[colunas_bool] = df_encoded_raca[colunas_bool].astype(int)

df_encoded_raca.head()

#### Sintomas e Comorbidades

Aqui é necessário transformar o 2 em 0 

In [ ]:
sint_com_cols = [
  "FEBRE",    # Sintomas
  "MIALGIA",
  "CEFALEIA",
  "EXANTEMA",
  "VOMITO",
  "NAUSEA",
  "DOR_COSTAS",
  "CONJUNTVIT",
  "ARTRITE",
  "ARTRALGIA",
  "PETEQUIA_N",
  "LEUCOPENIA",
  "LACO",
  "DOR_RETRO",

  "DIABETES",   # Comorbidades
  "HEMATOLOG",
  "HEPATOPAT",
  "RENAL",
  "HIPERTENSA",
  "ACIDO_PEPT",
  "AUTO_IMUNE",
]

In [ ]:
df_encoded_final = df_encoded_raca.copy()
df_encoded_final.head()

In [ ]:
df_encoded_final[sint_com_cols] = df_encoded_final[sint_com_cols].replace({2:0})
df_encoded_final[sint_com_cols] = df_encoded_final[sint_com_cols].astype(int)
df_encoded_final.head()

### Clusterização

In [ ]:
clustering_cols = [
  "CS_ESCOL_N", # Categoricos transformados
  "CS_SEXO_M",
  "CS_GESTANT_S",
  "CS_RACA_1",
  "CS_RACA_2",
  "CS_RACA_3",
  "CS_RACA_4",
  "CS_RACA_5",
  
  "FEBRE",    # Sintomas
  "MIALGIA",
  "CEFALEIA",
  "EXANTEMA",
  "VOMITO",
  "NAUSEA",
  "DOR_COSTAS",
  "CONJUNTVIT",
  "ARTRITE",
  "ARTRALGIA",
  "PETEQUIA_N",
  "LEUCOPENIA",
  "LACO",
  "DOR_RETRO",

  "DIABETES",   # Comorbidades
  "HEMATOLOG",
  "HEPATOPAT",
  "RENAL",
  "HIPERTENSA",
  "ACIDO_PEPT",
  "AUTO_IMUNE",
]

df_clustering = df_encoded_final[clustering_cols]
df_clustering.head()

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_clustering)

In [ ]:
inertias = []
for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)

plt.plot(range(2, 11), inertias, marker='o')
plt.title("Elbow Method")
plt.xlabel("Número de clusters")
plt.ylabel("Inércia")
plt.show()


# kmeans = KMeans(n_clusters=5, random_state=42) 
# labels_kmeans = kmeans.fit_predict(X_scaled)
# df_clustering["cluster"] = labels_kmeans

In [ ]:
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

# plt.figure(figsize=(8, 6))
# plt.scatter(X_pca[:, 0], X_pca[:, 1])
# plt.xlabel('PC1')
# plt.ylabel('PC2')
# plt.title('Visualização dos dados com PCA (2D)')
# plt.grid(True)
# plt.show()

# 3. Visualizar os dados em 3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(
    X_pca[:, 0], X_pca[:, 1], X_pca[:, 2],
    c='gray',
)

ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.set_zlabel("PC3")
ax.set_title("Visualização PCA em 3D")
plt.show()